In [1]:
import os
import re

In [2]:
# Assuming the current folder is the scripts folder which is
# one level deep from the parent, project folder.
current_directory = os.getcwd()
project_path = os.path.dirname(current_directory)
output_folder = "profiling_reports"  # folder name where all the profile reports reside

# Paths
profile_folder = os.path.join(project_path, output_folder)
# print(f"Profile folder: {profile_folder}")
key_summary = []

Profile folder: /Users/richardkindle/Library/CloudStorage/GoogleDrive-eldnikr@gmail.com/My Drive/DE Academy/Projects/Healthcare/profiling_reports


In [3]:
def parse_profile_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    try:
        start = lines.index('--- Describe (transposed) ---\n') + 2
    except ValueError:
        return None

    describe_lines = []
    for line in lines[start:]:
        if line.strip() == '':
            break
        describe_lines.append(line.rstrip('\n'))

    if len(describe_lines) < 2:
        return None

    header_line = describe_lines[0]
    data_lines = describe_lines[1:]
    max_lens = []

    for line in data_lines:
        cols = [col.strip() for col in line.split('  ') if col.strip()]
        for idx, val in enumerate(cols):
            if len(max_lens) <= idx:
                max_lens.append(len(val))
            else:
                max_lens[idx] = max(max_lens[idx], len(val))

    # for line in data_lines:
    #     cols = line.split('  ')
    #     # Get rid of empty cols
    #     actual_cols = []
    #     for col in cols:
    #         if len(col) > 0:
    #             actual_cols.append(col.strip())

    #     for idx, actual_col in enumerate(actual_cols):
            
    #         if len(max_lens) <= idx:
    #             max_lens.append(len(actual_col))
    #         else:
    #             col_len = len(actual_col)
    #             max_len = max_lens[idx]
    #             if col_len > max_len:
    #                 max_lens[idx] = col_len

    col_positions = [0]  # Initial value; column start position of zero
    
    # print(f"max_lens: {max_lens}")
    
    for idx, col_len in enumerate(max_lens):
        # 
        # if idx < len(max_lens) - 1:
        col_positions.append(col_len + col_positions[idx] + 2)

    # print(f"col_positions: {col_positions}")
    
    # Step 3: Build headers
    headers = ['column'] + re.split(r'\s{1,}', header_line.strip())
    # headers = ['column'] + [header_line[col_positions[i]:col_positions[i + 1]].strip()
    #                         for i in range(1, len(col_positions) - 1)]
    # print(f"Headers: {headers}")

    results = []
    for line in data_lines:
        fields = [line[col_positions[i]:col_positions[i + 1]].strip()
                  for i in range(0, len(col_positions) - 1)]
        # print(f"fields: {fields}")
        
        if len(fields) != len(headers):
            print(f"MISMATCH: number of fields does not match number of headers")
            continue
            
        row = dict(zip(headers, fields))
        # print(f"row: {row}")
        
        try:
            count = float(row.get('count', '0').replace(',', ''))
            unique = float(row.get('unique', '0').replace(',', ''))
        except ValueError:
            continue

        col_name = row['column']
        if count > 1 and count == unique:
            results.append((col_name, '✅ Likely PK'))
        elif count > 1 and unique / count >= 0.9:
            results.append((col_name, '⚠️ High uniqueness'))

    if not results:
        print(f"✅ Loaded: {os.path.basename(filepath)} — no candidate keys found")

    return results


In [4]:
# header_line = '                                            count unique                                                                 top    freq         mean           std    min       25%      50%      75%      max'
# col_positions = [0, 41, 51, 58, 126, 134, 147, 161, 168, 178, 187, 196, 205]
header_line = "                              count unique                               top freq        mean          std  min    25%    50%    75%     max"
max_lens = [28, 5, 3, 32, 3, 10, 11, 3, 5, 5, 5, 6]
col_positions = [0, 30, 37, 42, 76, 81, 93, 106, 111, 118, 125, 132, 140]

i = 2
# col = header_line[col_positions[i]:col_positions[i+1]-2]  #.strip()
# col = header_line[col_positions[i-1] + max_lens[i]:col_positions[i] + max_lens[i]]
header = ['column'] + re.split(r'\s{1,}', header_line.strip())
print(f"'{header}'")

'['column', 'count', 'unique', 'top', 'freq', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']'


In [5]:
# Loop through all profile files
primary_profile_file = "PBJ_Daily_Nurse_Staffing_Q2_2024_profile.txt"

for file in os.listdir(profile_folder):
    if (file.startswith('NH') and file.endswith('_profile.txt')) or file == primary_profile_file:
        print(f"\nInspecting file: {file}")
        filepath = os.path.join(profile_folder, file)
        candidates = parse_profile_file(filepath)
        if candidates:
            key_summary.append({
                'file': file.replace('_profile.txt', ''),
                'keys': [f"{col} ({note})" for col, note in candidates]
            })


Inspecting file: PBJ_Daily_Nurse_Staffing_Q2_2024_profile.txt
✅ Loaded: PBJ_Daily_Nurse_Staffing_Q2_2024_profile.txt — no candidate keys found

Inspecting file: NH_HealthCitations_Oct2024_profile.txt
✅ Loaded: NH_HealthCitations_Oct2024_profile.txt — no candidate keys found

Inspecting file: NH_CitationDescriptions_Oct2024_profile.txt

Inspecting file: NH_HlthInspecCutpointsState_Oct2024_profile.txt
✅ Loaded: NH_HlthInspecCutpointsState_Oct2024_profile.txt — no candidate keys found

Inspecting file: NH_CovidVaxProvider_20241027_profile.txt

Inspecting file: NH_ProviderInfo_Oct2024_profile.txt

Inspecting file: NH_StateUSAverages_Oct2024_profile.txt

Inspecting file: NH_CovidVaxAverages_20241027_profile.txt

Inspecting file: NH_QualityMsr_MDS_Oct2024_profile.txt
✅ Loaded: NH_QualityMsr_MDS_Oct2024_profile.txt — no candidate keys found

Inspecting file: NH_DataCollectionIntervals_Oct2024_profile.txt

Inspecting file: NH_FireSafetyCitations_Oct2024_profile.txt
✅ Loaded: NH_FireSafetyCita

In [6]:
# Output results
print("\n=== Candidate Key Columns Summary ===")
for entry in key_summary:
    print(f"\n{entry['file']}:")
    for key in entry['keys']:
        print(f"  - {key}")


=== Candidate Key Columns Summary ===

NH_CitationDescriptions_Oct2024:
  - Deficiency Prefix and Number (✅ Likely PK)
  - Deficiency Description (⚠️ High uniqueness)

NH_CovidVaxProvider_20241027:
  - CMS Certification Number (CCN) (✅ Likely PK)

NH_ProviderInfo_Oct2024:
  - CMS Certification Number (CCN) (✅ Likely PK)
  - Provider Name (⚠️ High uniqueness)
  - Provider Address (⚠️ High uniqueness)
  - Location (⚠️ High uniqueness)

NH_StateUSAverages_Oct2024:
  - State or Nation (✅ Likely PK)

NH_CovidVaxAverages_20241027:
  - State (✅ Likely PK)

NH_DataCollectionIntervals_Oct2024:
  - Measure Code (✅ Likely PK)
  - Measure Description (✅ Likely PK)
